<a href="https://colab.research.google.com/github/samyxdev/MyColabNotebooks/blob/main/imdb_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Embedding, Dense, Activation, LSTM
from tensorflow.keras.datasets import imdb

import os

from google.colab import drive

model_name = "lstm_imdb_model.h5"
gdrive_path = "/content/gdrive"
path = "MyDrive"

drive.mount(gdrive_path)

model_path = os.path.join(gdrive_path, path, model_name)
print("Chemin du model_path : ", model_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Chemin du model_path :  /content/gdrive/MyDrive/lstm_imdb_model.h5


In [22]:
maxf = 1000
maxlen = 80
((x_train, y_train), (x_test, y_test)) = imdb.load_data(num_words=maxf)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print(y_train[:5])

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

[1 0 0 1 0]


In [23]:
if os.path.exists(model_path):
  print("{} trouvé !".format(model_path))
  model = tf.keras.models.load_model(model_path)
  
  model.summary()

else:
  print("{} non trouvé ! Redefinition et entrainement ...".format(model_path))
  model = Sequential()
  model.add(Embedding(maxf, 128, input_length=maxlen))
  model.add(Dropout(0.2))

  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.2))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.summary()

  history = model.fit(x_train, y_train, epochs=5)
  model.save(model_path)
  print("Modèle enregistré dans ", model_path)


/content/gdrive/MyDrive/lstm_imdb_model.h5 trouvé !
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 128)           128000    
_________________________________________________________________
dropout (Dropout)            (None, 80, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 276,225
Trainable params: 276,225
Non-trainable params: 

In [24]:
score, acc = model.evaluate(x_test, y_test)

782/782 [==============================] - 24s 30ms/step - loss: 0.3801 - accuracy: 0.8292


In [30]:
def text_preprocessing(text, model_text_length=maxlen):
  text_dict = imdb.get_word_index()
  preform_text = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokenized_text = [text_dict[w] if w in text_dict.keys() else 0 for w in preform_text]

  # Pour compléter le texte afin d'atteindre la longueur requise
  return sequence.pad_sequences([tokenized_text], maxlen=model_text_length)

In [40]:
def pred_pipeline(str_text):
  processed_text = text_preprocessing(str_text)
  test_res = model.predict(processed_text)
  print("The text \"{}\" is likely to be {}. (score :{})".format(str_text, "positive" if test_res < 0.5 else "negative", test_res))


pred_pipeline("A very bad movie and bad actors")
pred_pipeline("A good movie and nice actors")


The text "A very bad movie and bad actors" is likely to be negative. (score :[[0.9292416]])
The text "A good movie and nice actors" is likely to be negative. (score :[[0.73992306]])
